In [389]:
import pandas as pd
import numpy as np
import os

In [390]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/SALT/Dashboards/')

<h1 style="color:blue;">
    Raw Scores
</h1>

In [391]:
dfp = pd.read_csv('./CBAScoredFiles/g3.csv')

/var/folders/0c/r0hw428532s7bttp9my0ym_00000gn/T/ipykernel_56343/1297672475.py:1: DtypeWarning: Columns (6,118) have mixed types. Specify dtype option on import or set low_memory=False.
  dfp = pd.read_csv('./CBAScoredFiles/g3.csv')


In [392]:
#check
dfp.shape

(363414, 145)

In [393]:
#df.to_pickle('./CBAScoredFiles/cba_data_for_dashboard.pkl')

In [394]:
#dfp = pd.read_pickle('./cba_data_for_dashboard.pkl')

In [407]:
unwanted = ('student_id','disecode','student_name','class','section','roll_no','unique_id','gender','social_category','medium_id'
            ,'Language_paperset','Language_jumble_set','Language_present_absent','LanguageTotal','LanguagePercentage_old','LanguagePercentage','Language_cheating_drop'
            ,'Maths_paperset','Maths_jumble_set','Maths_present_absent','MathsTotal','MathsPercentage_old','MathsPercentage','Maths_cheating_drop'
            ,'EVS_paperset','EVS_jumble_set','EVS_present_absent','EVSTotal','EVSPercentage_old','EVSPercentage','EVS_cheating_drop'
            ,'Science_paperset','Science_jumble_set','Science_present_absent','ScienceTotal','SciencePercentage_old','SciencePercentage','Science_cheating_drop'
            ,'SS_paperset','SS_jumble_set','SS_present_absent','SSTotal','SSPercentage_old','SSPercentage','SS_cheating_drop'
           )

In [408]:
var_list = list(dfp)

In [409]:
#EQ = EVS, MQ = Maths, LQ = Language (Telugu), SQ = Science, OQ = Social Science
prefixes = ('Score_EQ','Score_MQ','Score_LQ','Score_SQ','Score_OQ')
dropped = [x for x in var_list if x.startswith(prefixes)]

In [410]:
dfp.drop(dropped, axis = 1 , inplace = True)

In [ ]:
var_list = list(dfp)

In [411]:
retain = [ele for ele in var_list if ele not in unwanted]

In [412]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dimensions = ['student_id','disecode','student_name','class','section','roll_no'
                                  ,'unique_id','gender','social_category','medium_id'
                                  ,'Language_present_absent'
                                  ,'Maths_present_absent'
                                  ,'EVS_present_absent'
                                ]

In [413]:
dfp_unpivot = pd.melt(dfp,id_vars = dimensions
                        ,value_vars = retain)

In [414]:
#check
dfp_unpivot.shape

(20714598, 15)

In [415]:
dfp_unpivot[['Subject','Question']] = dfp_unpivot['variable'].str.split('Q',1,expand=True)

In [416]:
#Language mapped as Telugu for general files; English and Hindi will be separate scored files altogether
dfp_unpivot.loc[dfp_unpivot["Subject"]=="L","Subject"]="Telugu"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="M","Subject"]="Maths"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="E","Subject"]="EVS"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="O","Subject"]="SS"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="S","Subject"]="Science"

In [417]:
dfp_unpivot = dfp_unpivot.drop(['section','roll_no','student_id','variable'],axis=1)

In [418]:
dfp_unpivot.fillna('',inplace = True)

In [419]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
indexELM = dfp_unpivot[(((dfp_unpivot['Language_present_absent']=='A')|(dfp_unpivot['Language_present_absent']==''))&(dfp_unpivot['Subject']=='Telugu'))|
                       (((dfp_unpivot['Maths_present_absent']=='A')|(dfp_unpivot['Maths_present_absent']==''))&(dfp_unpivot['Subject']=='Maths'))|
                       (((dfp_unpivot['EVS_present_absent']=='A')|(dfp_unpivot['EVS_present_absent']==''))&(dfp_unpivot['Subject']=='EVS'))
                      ].index

dfp_unpivot.drop(indexELM, inplace=True)

In [420]:
#check
dfp_unpivot.shape

(20259966, 13)

In [421]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dfp_unpivot = dfp_unpivot.drop(['Language_present_absent','Maths_present_absent','EVS_present_absent'],axis=1)

In [422]:
dfp_unpivot['Question'] = dfp_unpivot['Question'].astype(np.int64)

In [423]:
dfp_unpivot = dfp_unpivot.rename(columns = {'unique_id':'Child_Id','disecode':'Udise'\
                                            ,'student_name':'Child_Name','class':'Grade'\
                                            ,'gender':'Gender','social_category':'Social_Category'\
                                            ,'medium_id':'Medium_Id','value':'Raw_Score','Subject':'Subject'\
                                            ,'Question':'Question_no'})

In [424]:
dfp_unpivot.to_pickle('./CBAScoredFiles/g3_scores.pkl')

<h1 style="color:blue;">
    Total Scores
</h1>

In [425]:
dfp = pd.read_csv('./CBAScoredFiles/g3.csv')

/var/folders/0c/r0hw428532s7bttp9my0ym_00000gn/T/ipykernel_56343/1297672475.py:1: DtypeWarning: Columns (6,118) have mixed types. Specify dtype option on import or set low_memory=False.
  dfp = pd.read_csv('./CBAScoredFiles/g3.csv')


In [426]:
#check
dfp.shape

(363414, 145)

In [437]:
unwanted = ('student_id','disecode','student_name','class','section','roll_no','unique_id','gender','social_category','medium_id'
            ,'Language_paperset','Language_jumble_set','Language_present_absent','LanguageTotal','LanguagePercentage_old','LanguagePercentage','Language_cheating_drop'
            ,'Maths_paperset','Maths_jumble_set','Maths_present_absent','MathsTotal','MathsPercentage_old','MathsPercentage','Maths_cheating_drop'
            ,'EVS_paperset','EVS_jumble_set','EVS_present_absent','EVSTotal','EVSPercentage_old','EVSPercentage','EVS_cheating_drop'
            ,'Science_paperset','Science_jumble_set','Science_present_absent','ScienceTotal','SciencePercentage_old','SciencePercentage','Science_cheating_drop'
            ,'SS_paperset','SS_jumble_set','SS_present_absent','SSTotal','SSPercentage_old','SSPercentage','SS_cheating_drop'
           )

In [438]:
var_list = list(dfp)

In [439]:
#EQ = EVS, MQ = Maths, LQ = Language (Telugu), SQ = Science, OQ = Social Science
prefixes = ('EQ','MQ','LQ','SQ','OQ')
dropped = [x for x in var_list if x.startswith(prefixes)]

In [440]:
dfp.drop(dropped, axis = 1 , inplace = True)

In [441]:
var_list = list(dfp)

In [442]:
retain = [ele for ele in var_list if ele not in unwanted]

In [443]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dimensions = ['student_id','disecode','student_name','class','section','roll_no'
                                  ,'unique_id','gender','social_category','medium_id'
                                  ,'Language_present_absent'
                                  ,'Maths_present_absent'
                                  ,'EVS_present_absent'
                                ]

In [444]:
dfp_unpivot = pd.melt(dfp,id_vars = dimensions
                        ,value_vars = retain)

In [445]:
dfp_unpivot[['S','Q']] = dfp_unpivot['variable'].str.split('_',1,expand=True)

In [446]:
dfp_unpivot[['Subject','Question']] = dfp_unpivot['Q'].str.split('Q',1,expand=True)

In [447]:
#Language mapped as Telugu for general files; English and Hindi will be separate scored files altogether
dfp_unpivot.loc[dfp_unpivot["Subject"]=="L","Subject"]="Telugu"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="M","Subject"]="Maths"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="E","Subject"]="EVS"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="O","Subject"]="SS"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="S","Subject"]="Science"

In [448]:
dfp_unpivot = dfp_unpivot.drop(['section','roll_no','student_id','variable','S','Q'],axis=1)

In [449]:
dfp_unpivot.fillna('',inplace = True)

In [450]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
indexELM = dfp_unpivot[(((dfp_unpivot['Language_present_absent']=='A')|(dfp_unpivot['Language_present_absent']==''))&(dfp_unpivot['Subject']=='Telugu'))|
                       (((dfp_unpivot['Maths_present_absent']=='A')|(dfp_unpivot['Maths_present_absent']==''))&(dfp_unpivot['Subject']=='Maths'))|
                       (((dfp_unpivot['EVS_present_absent']=='A')|(dfp_unpivot['EVS_present_absent']==''))&(dfp_unpivot['Subject']=='EVS'))
                      ].index

dfp_unpivot.drop(indexELM, inplace=True)

In [451]:
#check
dfp_unpivot.shape

(20259966, 13)

In [452]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dfp_unpivot = dfp_unpivot.drop(['Language_present_absent','Maths_present_absent'],axis=1)

In [453]:
dfp_unpivot['Question'] = dfp_unpivot['Question'].astype(np.int64)

In [454]:
dfp_unpivot = dfp_unpivot.rename(columns = {'unique_id':'Child_Id','disecode':'Udise'\
                                            ,'student_name':'Child_Name','class':'Grade'\
                                            ,'gender':'Gender','social_category':'Social_Category'\
                                            ,'medium_id':'Medium_Id','value':'Total_Score'\
                                            ,'Subject':'Subject','Question':'Question_no'})

In [455]:
dfp_unpivot.to_pickle('CBAScoredFiles/g3_raw_scores.pkl')

In [456]:
dfp_unpivot = pd.DataFrame()

<h1 style="color:blue;">
    Merge files
</h1>

In [457]:
Scores = pd.read_pickle('./CBAScoredFiles/g3_scores.pkl')

In [458]:
Raw = pd.read_pickle('./CBAScoredFiles/g3_raw_scores.pkl')

In [459]:
df = pd.merge(Scores, Raw, on=['Child_Id','Udise','Child_Name','Grade','Gender','Social_Category'\
                               ,'Medium_Id','Subject','Question_no'], suffixes=("", ""))


<h1 style="color:blue;">
    School mapping
</h1>

In [460]:
school_mapping = pd.read_excel('./MappingFiles/AP_SchoolMaster_DistrictMandal_Mapping.xlsx')

In [461]:
df = pd.merge(df,school_mapping,how='left',left_on='Udise',right_on='SCHOOLCODE')

In [462]:
df = df.assign(State='Andhra Pradesh',Topic='',Subtopic='',Skill='',MUA='')

In [463]:
df = df.rename(columns={'SCHOOLNAME':'School','schooltype':'School_Type','Udise':'Udise'\
                                       ,'Social_Category':'Social_Category','District ':'District'})

In [464]:
df = df.drop(['SCHOOLCODE','Medium_Id'],axis=1)

In [465]:
df = df[[
'State',
'District',
'Mandal',
'School',
'Udise',
'School_Type',
'Grade',
'Child_Name',
'Child_Id',
'Gender',
'Social_Category',
'Subject',
'Question_no',
'Total_Score',
'Raw_Score',
'Topic',
'Subtopic',
'Skill',
'MUA']]

In [466]:
# df.to_csv('./CBALongFiles/CBA_G3_LongFile.csv')

In [ ]:
df.to_pickle('./CBALongPickleFiles/G3.pkl')